# 円周率計算で酷評されていたPythonプログラムの高速化を試みる

## Base Case：とにかく遅いと指摘されていたプログラム

Python, JavaScript, VBAで同じアルゴリズムを用いて円周率を求めるプログラムを書いたとき、Pythonは最も実行速度が遅かったので酷評されていた。

- JavaScriptとは比較にならないほど遅い。
- VBAよりも遅い。

まあ、Pythonのループは遅いからね。
「Pythonでforループ書いたら負け」なんて思っていた時期もありました。
書きやすいけど、そもそも速い言語じゃないしね。
ちょっと逸れるけど、Python3.11から高速化に向けて動いているので期待してます。

でもまあ、酷評していた方の言いたいことは理解できます。

で、これが酷評されいたものとほぼ同じコード。
モンテカルロ法の乱数は閉区間`[0, 1]`だろ、これじゃ半開区間`[0, 1)`じゃんって議論はこの検証の本質から外れるので置いておきますね。

でも、この書き方ってPythonicじゃないってのは置いておいて、Pythonでも速くする方法あるのにな。
あとね、他の言語と共通しちゃうけど、そもそもアルゴリズムにも問題があるような気が。。。

In [1]:
%%timeit -n1 -r3

r = 10000
count = 0

for x in range(r):
    for y in range(r):
        if pow(x, 2) + pow(y, 2) <= pow(r, 2):
            count += 1

pi = 4 * count / pow(r, 2)
print(pi)

3.14199048
3.14199048
3.14199048
1min 16s ± 2.36 s per loop (mean ± std. dev. of 3 runs, 1 loop each)


## Case1：アルゴリズムは変えないで、一番手っ取り早い*「関数化してnumba」*を試す

`numba`で高速化がうまくいけばラッキーと思っていざ実践。

### 結果：めちゃくちゃ速くなった。

In [2]:
import numpy as np
from numba import jit, njit

In [20]:
%%timeit -n1 -r10

@jit
def calc_pi():
    r = 10000
    count = 0

    for x in range(r):
        for y in range(r):
            if pow(x, 2) + pow(y, 2) <= pow(r, 2):
                count += 1
    
    return 4 * count / pow(r, 2)

print(calc_pi())

3.14199048
3.14199048
3.14199048
3.14199048
3.14199048
3.14199048
3.14199048
3.14199048
3.14199048
3.14199048
129 ms ± 5.2 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


---
## ちょっとだけ`numba`について備忘録
---

### Memo1：numbaのコンパイルエラーで止めちゃえ

numbaはPythonのJust-In-Timeコンパイラで、numpyの配列や関数、ループを使ったコードで最もよく機能します。
（[numba公式ドキュメント](https://numba.pydata.org/numba-doc/dev/user/5minguide.html)より引用）

numba関連で頻出するJITはJust-In-Timeのことだね。

公式には最もよく機能するってあるけど、実は何でもかんでも高速化できるわけじゃない。

numbaが対応していないPythonの機能はたくさんあります。
numba非対応の機能を`@jit`だけでデコレートすると元のコードより遅くなることもあるよ。
だから、取り敢えず`@jit`の精神はここで捨てよう。理由は以下の通り。

1. numbaが`@jit`でデコレートされた箇所をJITコンパイルする。
1. コンパイルするのだから、コンパイラに無い機能を使えば当然コンパイルエラーを起こすよ。
1. エラーを起こしたコードはオブジェクトモードっていうたまーに高速化するけど、失敗したらただのPythonねっていうモードでコンパイルされるよ。そのおかげでコードは止まることなく動くよ。
1. 結局、コンパイル ---> 失敗 ---> オブジェクトモードコンパイル ---> Pythonの順で動くから、当然遅くなる。

じゃあ、コンパイルに失敗したら止めればいんじゃね？？？
そうです。その機能あります。

`nopython=True`ってオプションでコンパイルに失敗した時点で止められます！

だから`@jit`は止めて`@jit(nopython=True)`を使いましょう！

In [4]:
%%timeit -n1 -r10

@jit(nopython=True)
def calc_pi():
    r = 10000
    count = 0

    for x in range(r):
        for y in range(r):
            if pow(x, 2) + pow(y, 2) <= pow(r, 2):
                count += 1
    
    return 4 * count / pow(r, 2)

print(calc_pi())

3.14199048
3.14199048
3.14199048
3.14199048
3.14199048
3.14199048
3.14199048
3.14199048
3.14199048
3.14199048
130 ms ± 6.54 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


### Memo2：`@jit(nopython=True)`って書くのめんどくせー

長いタイピングってめんどくさいよね。
実は`@jit(nopython=True)`のエイリアスがあります。

`@njit`

これはめっちゃ使う機能ですね。

実務でアルゴリズムを練る開発時間は無いけど、ホットスポットの実行時間は高速化したい。

こんなときは、サクッと`@njit`を試してみる価値はあるよ。


In [5]:
%%timeit -n1 -r10

@njit
def calc_pi():
    r = 10000
    count = 0

    for x in range(r):
        for y in range(r):
            if pow(x, 2) + pow(y, 2) <= pow(r, 2):
                count += 1
    
    return 4 * count / pow(r, 2)

print(calc_pi())

3.14199048
3.14199048
3.14199048
3.14199048
3.14199048
3.14199048
3.14199048
3.14199048
3.14199048
3.14199048
131 ms ± 6.83 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


### Memo3：静的型付けコンパイルを使いこなそう

JITコンパイルの実行速度が速いのは静的型付けコンパイルをしてるからなんですよ。

あれ？どこかで型宣言したっけ？
そうです、してません。
numbaが優秀なのは型推論してくれるところなんです。

だから、引数と戻り値は型宣言した方が高速化の恩恵は受けやすくなるよ。

また、`@njit`だけだとコンパイルエラーを起こすこと（np.emptyを内部で使うとエラーが発生するのが有名）があるけど、引数と戻り値を型宣言することでエラーを起こさなくなるコードもあるから型宣言はできるだけ使おう。

型は`@njit("戻り値型(第1引数型, 第2引数型, ...)")`で書くよ。ちなみに、`@njit("f8(i4, i4)")`と`@njit("float64(int32, int32)")`は等価だよ。

配列を使う場合は`@njit("f8[:](f8[:, :], f8[:, :, :])")`みたいな書き方で宣言できるよ。

戻り値が複数あるときは`@njit("Tuple((i8, i8))(i8, i8)")`みたいな書き方で大丈夫。

主な型を[公式ドキュメント](https://numba.readthedocs.io/en/stable/reference/types.html#basic-types)から抜粋するね。


| Type name(s) | Shorthand | Comments |
| :----------- | :-------: | -------- |
| boolean | b1 | バイトで表す |
| uint8, byte | u1 | 8ビット符号なしバイト |
| uint16 | u2 | 16ビット符号なし整数 |
| uint32 | u4 | 32ビット符号なし整数 |
| uint64 | u8 | 64ビット符号なし整数 |
| int8, char | i1 | 8ビット符号付きバイト |
| int16 | i2 | 16ビット符号付き整数 |
| int32 | i4 | 32ビット符号付き整数 |
| int64 | i8 | 64ビット符号付き整数 |
| float32 | f4 | 単精度浮動小数点数 |
| float64, double | f8 | 倍精度浮動小数点数 |
| complex64 | c8 | 単精度複素数 |
| complex128 | c16 | 倍精度複素数 |

In [6]:
%%timeit -n1 -r10

@njit("f8()")
def calc_pi():
    r = 10000
    count = 0

    for x in range(r):
        for y in range(r):
            if pow(x, 2) + pow(y, 2) <= pow(r, 2):
                count += 1
    
    return 4 * count / pow(r, 2)

print(calc_pi())

3.14199048
3.14199048
3.14199048
3.14199048
3.14199048
3.14199048
3.14199048
3.14199048
3.14199048
3.14199048
131 ms ± 3.58 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


#### numba関連の参考文献

- [numba 公式ドキュメント](https://numba.readthedocs.io/en/stable/index.html)
- [numbaとnumpyで速いループ処理を書くためのガイド](https://www.haya-programming.com/entry/2020/04/09/062624)
- [numbaによるfor文の高速化とjitの引数](https://qiita.com/nabenabe0928/items/a02964d8b48619b1e348)
- [PythonとNumbaで数値計算を高速化するときの知見](https://qiita.com/sonokr/items/4b35ac2137bb16e82bd0#2-2-%E8%A4%87%E6%95%B0%E3%81%AE%E6%88%BB%E3%82%8A%E5%80%A4%E3%82%92%E8%BF%94%E3%81%99)
- [numbaでざっくりPython高速化](https://qiita.com/gyu-don/items/9d223b007ca620e95abc)




---
## ここで終わったらつまらないので色々と試す
---

## Case2：Pythonでpowって使わないよね？？

powってPythonで初めて見たような気が。
知識不足なだけかも。
でも、数値計算するときは普通べき乗計算は`**`で書くよね？？

`pow`って速いの？
普通にべき乗計算した。

### 結果：少しだけ速くなった。

In [7]:
%%timeit -n1 -r3

r = 10000
count = 0

for x in range(r):
    for y in range(r):
        if x**2 + y**2 <= r**2:
            count += 1

pi = 4 * count / r**2
print(pi)

3.14199048
3.14199048
3.14199048
1min 5s ± 336 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)


## Case3：なんでループ内で無意味な計算しているの？？

ループ内で`r**2`って計算する必要ないよね？？ 
値が変わらないのだから、ループ内で処理するだけ無駄だよね？

`r**2`を事前に計算し、ループ内での計算回数をゼロにした。

### 結果：少しだけ速くなった。

In [8]:
%%timeit -n1 -r3

r = 10000
count = 0

r_squared = r**2

for x in range(r):
    for y in range(r):
        if x**2 + y**2 <= r_squared:
            count += 1

pi = 4 * count / r_squared
print(pi)

3.14199048
3.14199048
3.14199048
46.3 s ± 78.2 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)


## Case4：またまたnumbaを使ってみる

もとのプログラムより少し速くなったので、ちょっと欲がでてまたまたnumbaを使ってみる。

確かにCase1の初回実行と今回の初回実行の比較では少しだけ速いかも？？
キャッシュするから2回目以降の違いはわからない。。。

### 結果：Case1と比較して効果があるのかわからない。

In [9]:
%%timeit -n1 -r10

@njit("f8()")
def calc_pi():
    r = 10000
    count = 0

    r_squared = r**2

    for x in range(r):
        for y in range(r):
            if x**2 + y**2 <= r_squared:
                count += 1
    
    return 4 * count / r_squared

print(calc_pi())

3.14199048
3.14199048
3.14199048
3.14199048
3.14199048
3.14199048
3.14199048
3.14199048
3.14199048
3.14199048
130 ms ± 4.78 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


## Case5：内包表記を試してみる　Part１

`for`よりも`内包表記`の方が速いってのがPythonの常識？だよね。

けど、内包表記で`if ... else`を書くと遅くなるのは知ってる。。。

### 結果 ---> forの二重ループとほとんど変わらない。

In [10]:
%%timeit -n1 -r3

r = 10000

r_squared = r**2

count = sum([1 if x**2 + y**2 <= r_squared else 0 for x in range(r) for y in range(r)])

pi = 4 * count / r_squared
print(pi)

3.14199048
3.14199048
3.14199048
47.2 s ± 1.13 s per loop (mean ± std. dev. of 3 runs, 1 loop each)


## Case6：内包表記を試してみる　Part2

内包表記の`if ... else`オメーはダメだ。

だから、内包表記の条件は`if`だけにして`else`は追い出したぜ。
あれ？？？

### 結果 ---> ほとんど変わらない。


In [11]:
%%timeit -n1 -r3

r = 10000

r_squared = r**2

count = sum([1 for x in range(r) for y in range(r) if x**2 + y**2 <= r_squared])

pi = 4 * count / r_squared
print(pi)

3.14199048
3.14199048
3.14199048
45.3 s ± 314 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)


## Case7：内包表記を試してみる　Part3

内包表記の`if`もダメだった。

じゃあ`if`も取り除いて`True`のみが返ってくるようにしてやるよ。
やってること`if`文と変わらないから効果あるか知らんけど。

### 結果 ---> ほとんど変わらない。

In [12]:
%%timeit -n1 -r3

r = 10000

r_squared = r**2

count = sum([x**2 + y**2 <= r_squared for x in range(r) for y in range(r)])

pi = 4 * count / r_squared
print(pi)

3.14199048
3.14199048
3.14199048
46.3 s ± 152 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)


## Case8：べき乗計算が遅い？？ベクトル計算じゃ

べき乗計算がホットスポットならば、`numpy`でべき乗配列を作れば高速化するかも？

`numpy`さん、いよいよベクトル計算の出番です。
`np.arrange(r)**2`で値が二乗された配列を作りました。

### 結果 ---> 速くなった。

In [13]:
%%timeit -n1 -r3
 
r = 10000
count = 0

r_squared = r**2
squared_array = np.arange(r)**2

for x2 in squared_array:
    for y2 in squared_array:
        if x2 + y2 <= r_squared:
            count += 1

pi = 4 * count / r_squared
print(pi)

3.14199048
3.14199048
3.14199048
23 s ± 88.1 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)


## Case9：内包表記再び

速くはなったけど、やっぱり遅くなっている原因って二重ループが主犯だよね？

重い処理のべき乗計算だけ`numpy`のベクトル計算に任せて、ループは内包表記で無くしてみよう。

### 結果 ---> 速くなった。

In [14]:
%%timeit -n1 -r3

r = 10000

r_squared = r**2
squared_array = np.arange(r)**2
squared_list = squared_array.tolist()

count = sum([x2 + y2 <= r_squared for x2 in squared_list for y2 in squared_list])

pi = 4 * count / r_squared
print(pi)

3.14199048
3.14199048
3.14199048
8.18 s ± 87.6 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)


## Case10：横着したら大爆死

二重ループを消すためにわざわざnumpy配列をリストに戻したけど、これって意味あるのかな？

実務では使った記憶がないnumpy配列を内表表記で回すって方法を試してみよう。リストに戻すのもめんどうだしね。

### 結果 ---> 大爆死。いやいや最初のプログラムより遅いって大悪手だな。

In [15]:
%%timeit -n1 -r3

r = 10000

r_squared = r**2
squared_array = np.arange(r)**2

count = sum([x2 + y2 <= r_squared for x2 in squared_array for y2 in squared_array])

pi = 4 * count / r_squared
print(pi)

3.14199048
3.14199048
3.14199048
3min 11s ± 2.02 s per loop (mean ± std. dev. of 3 runs, 1 loop each)


In [16]:
%%timeit -n1 -r10

@njit("f8()")
def calc_pi():
    r = 10000
    count = 0

    r_squared = r**2
    squared_array = np.arange(r)**2

    for x2 in squared_array:
        for y2 in squared_array:
            if x2 + y2 <= r_squared:
                count += 1

    return 4 * count / r_squared

print(calc_pi())

3.14199048
3.14199048
3.14199048
3.14199048
3.14199048
3.14199048
3.14199048
3.14199048
3.14199048
3.14199048
252 ms ± 122 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


## Case12：numpyの本気？

全部numpyのベクトル計算を利用すれば速いんじゃ？いよいよ数値計算の真打登場でいいのかな？

この問題を与えられたら、自分なら最初この方法で書いてみるかも。。。

1. 連続した値の二乗配列を作る。
1. 二乗配列をリスト変換しr倍することで連続した値の二乗をr回繰り返した1次元リストを作る。 
    - リスト変換しないで配列のままr倍すると配列の要素自体をr倍するベクトル計算になってしまう。
1. リストの要素数は $ r \times r $ なのでリシェイプして格子状にする。
1. これで格子のX方向の値は埋まった。
1. 格子のY方向の値は上記を転置して対応する。
1. X方向、Y方向の格子の値を足して、半径の二乗以下となったケースをカウントする。

### 結果 ---> めっちゃ速いでいいのかな？

In [17]:
%%timeit -n1 -r3
r = 10000

r_squared = r**2
squared_array = np.arange(r)**2

x2_grid = np.array([squared_array] * r).reshape(r, r)
count = np.sum((x2_grid + x2_grid.T) <= r_squared)

pi = 4 * count / r_squared
print(pi)

3.14199048
3.14199048
3.14199048
1.28 s ± 9.35 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)


## Case13：np.tile()って便利なものがあった

numpyには配列をタイル状に繰り返し並べて配列を作る便利な関数があった。。。知りませんでした。

その名は`np.tile()`です。書くのは楽になるけど、速くなるのかな？

### 結果 ---> 



In [18]:
%%timeit -n1 -r3
r = 10000

r_squared = r**2
squared_array = np.arange(r)**2

x2_grid = np.tile(squared_array, (r, 1))
count = np.sum((x2_grid + x2_grid.T) <= r_squared)

pi = 4 * count / r_squared
print(pi)

3.14199048
3.14199048
3.14199048
1.28 s ± 2.42 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)
